# Data Integration Data Model

In [0]:
from functools import reduce
from pyspark.sql import SparkSession, Row, functions as F
from pyspark.sql.types import (
    StructField
    , StringType
    , IntegerType
    , DoubleType
    , BooleanType
    , StructType
)
spark = SparkSession.builder.appName("Pipeline_Test").getOrCreate()

### Aims

The goal is to seek an algorithmic approach to merging the following datasets where the column names are implicitly defined based on domain knowledge and EDA:

In [0]:
dataset_A = {
    1: (1769, 6421.79, 0, 'Dave', 30)
    , 2: (8341, 1137.20, 1, 'Bob', 42)
    , 3: (4507, 911.93, 0, 'Jess', 21)
    , 4: (None, 1234.0, 0, 'Lisa', 37)
    , 5: (4405, 7891.37, None, 'Andy', 38)
    , 6: (6666, 1234.0, 0, None, 99)
}

dataset_B = {
    0: (1769, 30, 6421.79, 0, None)
    , 1: (4507, 21, 911.93, 0, 48)
    , 2: (4405, 38, 7891.37, 1, 587)
    , 3: (8341, 42, 10.56, 1, 999)
    , 4: (7777, 56, 4939.32, None, 1233)
    , 5: (6666, 98, 1234.0, 0, 1234)
}

dataset_C = {
    4232: (None, 58.58, 23)
    , 9947: (687, 3400.21, 66)
    , 8888: (951, 78.29, 18)
    , 6666: (543, 435.0, 97)
}

##### Step 0

A target for the data is set based off EDA.

In [0]:
target_df = spark.createDataFrame([], schema=StructType(fields=[
    StructField('cust_id', IntegerType(), True)
    , StructField('name', StringType(), True)
    , StructField('age', IntegerType(), True)
    , StructField('balance', DoubleType(), True)
    , StructField('c_score', IntegerType(), True)
    , StructField('churn', IntegerType(), True)
]))
target_df.printSchema()

root
 |-- cust_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- balance: double (nullable = true)
 |-- c_score: integer (nullable = true)
 |-- churn: integer (nullable = true)



##### Step 1

It is assumed
- `cust_id` is consistent across all datasets.
- all datasets have similar data: ie the columns are somewhat the same.
- the schemas are roughly compatible: eg age is numeric instead of a mix of string/numeric.

Each dataframe attribute is labeled according to the data because it is not assumed that the data is consistent. Care must be taken to validate consistency when merging multiple data sources into the target table.

In [0]:
df_A = spark.createDataFrame([
    Row(
        idx=i
        , cust_id=r[0]
        , balance_A=r[1]
        , churn_A=r[2]
        , name_A=r[3]
        , age_A=r[4]
    ) for i,r in dataset_A.items()
])

df_B = spark.createDataFrame([
    Row(
        idx=i
        , cust_id=r[0]
        , age_B=r[1]
        , balance_B=r[2]
        , churn_B=r[3]
        , c_score_B=r[4]
    ) for i,r in dataset_B.items()
])

df_C = spark.createDataFrame([
    Row(
        cust_id=i
        , c_score_C=r[0]
        , balance_C=r[1]
        , age_C=r[2]
    ) for i,r in dataset_C.items()
])

In [0]:
df_A.show()

+---+-------+---------+-------+------+-----+
|idx|cust_id|balance_A|churn_A|name_A|age_A|
+---+-------+---------+-------+------+-----+
|  1|   1769|  6421.79|      0|  Dave|   30|
|  2|   8341|   1137.2|      1|   Bob|   42|
|  3|   4507|   911.93|      0|  Jess|   21|
|  4|   null|   1234.0|      0|  Lisa|   37|
|  5|   4405|  7891.37|   null|  Andy|   38|
|  6|   6666|   1234.0|      0|  null|   99|
+---+-------+---------+-------+------+-----+



In [0]:
df_B.show()

+---+-------+-----+---------+-------+---------+
|idx|cust_id|age_B|balance_B|churn_B|c_score_B|
+---+-------+-----+---------+-------+---------+
|  0|   1769|   30|  6421.79|      0|     null|
|  1|   4507|   21|   911.93|      0|       48|
|  2|   4405|   38|  7891.37|      1|      587|
|  3|   8341|   42|    10.56|      1|      999|
|  4|   7777|   56|  4939.32|   null|     1233|
|  5|   6666|   98|   1234.0|      0|     1234|
+---+-------+-----+---------+-------+---------+



In [0]:
df_C.show()

+-------+---------+---------+-----+
|cust_id|c_score_C|balance_C|age_C|
+-------+---------+---------+-----+
|   4232|     null|    58.58|   23|
|   9947|      687|  3400.21|   66|
|   8888|      951|    78.29|   18|
|   6666|      543|    435.0|   97|
+-------+---------+---------+-----+



##### Step 2

It is ensured that each customer has a `cust_id`.

In [0]:
df_A = df_A.filter(df_A.cust_id.isNotNull())
df_B = df_B.filter(df_B.cust_id.isNotNull())
df_C = df_C.filter(df_C.cust_id.isNotNull())

In [0]:
df_A.show()

+---+-------+---------+-------+------+-----+
|idx|cust_id|balance_A|churn_A|name_A|age_A|
+---+-------+---------+-------+------+-----+
|  1|   1769|  6421.79|      0|  Dave|   30|
|  2|   8341|   1137.2|      1|   Bob|   42|
|  3|   4507|   911.93|      0|  Jess|   21|
|  5|   4405|  7891.37|   null|  Andy|   38|
|  6|   6666|   1234.0|      0|  null|   99|
+---+-------+---------+-------+------+-----+



##### Step 3

It is assumed that `cust_id`
- exists.
- is unique.

That is, `cust_id` is assumed to be a valid primary key.

In [0]:
full_joined_df = df_A.join(df_B, on='cust_id', how='full').join(df_C, on='cust_id', how='full')

##### Step 4a (Observation)

Data validation. To find an idea on how to approach this, note that data in `full_joined_df` can be validated row-wise.

In [0]:
full_joined_df.show()

+-------+----+---------+-------+------+-----+----+-----+---------+-------+---------+---------+---------+-----+
|cust_id| idx|balance_A|churn_A|name_A|age_A| idx|age_B|balance_B|churn_B|c_score_B|c_score_C|balance_C|age_C|
+-------+----+---------+-------+------+-----+----+-----+---------+-------+---------+---------+---------+-----+
|   1769|   1|  6421.79|      0|  Dave|   30|   0|   30|  6421.79|      0|     null|     null|     null| null|
|   4232|null|     null|   null|  null| null|null| null|     null|   null|     null|     null|    58.58|   23|
|   4405|   5|  7891.37|   null|  Andy|   38|   2|   38|  7891.37|      1|      587|     null|     null| null|
|   4507|   3|   911.93|      0|  Jess|   21|   1|   21|   911.93|      0|       48|     null|     null| null|
|   6666|   6|   1234.0|      0|  null|   99|   5|   98|   1234.0|      0|     1234|      543|    435.0|   97|
|   7777|null|     null|   null|  null| null|   4|   56|  4939.32|   null|     1233|     null|     null| null|
|

Each attribute can be selected, and each valid row will have one non-null value at most. Rows with more than one non-null value is a data clash, and must be dropped.

In [0]:
full_joined_df.select(['cust_id']+[c for c in full_joined_df.columns if 'balance' in c]).withColumn(
    'distinct_count',
    F.size(F.array_distinct(
        F.expr('filter(array(balance_A, balance_B, balance_C), x -> x IS NOT NULL)')
    ))
).show()

+-------+---------+---------+---------+--------------+
|cust_id|balance_A|balance_B|balance_C|distinct_count|
+-------+---------+---------+---------+--------------+
|   1769|  6421.79|  6421.79|     null|             1|
|   4232|     null|     null|    58.58|             1|
|   4405|  7891.37|  7891.37|     null|             1|
|   4507|   911.93|   911.93|     null|             1|
|   6666|   1234.0|   1234.0|    435.0|             2|
|   7777|     null|  4939.32|     null|             1|
|   8341|   1137.2|    10.56|     null|             2|
|   8888|     null|     null|    78.29|             1|
|   9947|     null|     null|  3400.21|             1|
+-------+---------+---------+---------+--------------+




Searching `full_joined_df`, it is seen that customers `8341` and `6666` have outliers in their balances. Counting distinct values can be done for each attribute, and customers can be validated if all the distinct counts are 0 or 1.

##### Step 4b (Implementation)

Now the observation is implemented.

In [0]:
outer_joined = full_joined_df

Count the values.

In [0]:
for column_name in target_df.columns[1:]:
    columns = [c for c in outer_joined.columns if column_name in c]
    quoted_columns = [f"`{c}`" for c in columns]
    temp = ', '.join(quoted_columns)
    distinct_count = F.size(
        F.array_distinct(
            F.expr(f"filter(array({temp}), x -> x IS NOT NULL)")
        )
    )
    outer_joined = outer_joined.withColumn(
        f'valid_{column_name}',
        F.when(distinct_count > 1, F.lit(False))
         .otherwise(F.lit(True))
    )

Filter for valid rows.

In [0]:
boolean_columns = [c for c in outer_joined.columns if 'valid' in c]
filter_cond = (F.col(column) == True for column in boolean_columns)
filter_expr = reduce(lambda x, y: x & y, filter_cond)
outer_joined = outer_joined.filter(filter_expr)

In [0]:
outer_joined.show()

+-------+----+---------+-------+------+-----+----+-----+---------+-------+---------+---------+---------+-----+----------+---------+-------------+-------------+-----------+
|cust_id| idx|balance_A|churn_A|name_A|age_A| idx|age_B|balance_B|churn_B|c_score_B|c_score_C|balance_C|age_C|valid_name|valid_age|valid_balance|valid_c_score|valid_churn|
+-------+----+---------+-------+------+-----+----+-----+---------+-------+---------+---------+---------+-----+----------+---------+-------------+-------------+-----------+
|   1769|   1|  6421.79|      0|  Dave|   30|   0|   30|  6421.79|      0|     null|     null|     null| null|      true|     true|         true|         true|       true|
|   4232|null|     null|   null|  null| null|null| null|     null|   null|     null|     null|    58.58|   23|      true|     true|         true|         true|       true|
|   4405|   5|  7891.37|   null|  Andy|   38|   2|   38|  7891.37|      1|      587|     null|     null| null|      true|     true|         

Construct the target dataframe by coalescing values from the filtered outerjoined table.

In [0]:
target_schema = {field.name: field.dataType for field in target_df.schema.fields}
for column_name in target_df.columns[1:]:
    matching_columns = [c for c in outer_joined.columns if c.startswith(column_name)]
    casted_columns = [F.col(c).cast(target_schema[column_name]) for c in matching_columns]
    outer_joined = outer_joined.withColumn(column_name, F.coalesce(*casted_columns))
final_columns = [F.col(c) for c in target_df.columns]
final_df = outer_joined.select(*final_columns)

##### Results

In [0]:
final_df.show()

+-------+----+---+-------+-------+-----+
|cust_id|name|age|balance|c_score|churn|
+-------+----+---+-------+-------+-----+
|   1769|Dave| 30|6421.79|   null|    0|
|   4232|null| 23|  58.58|   null| null|
|   4405|Andy| 38|7891.37|    587|    1|
|   4507|Jess| 21| 911.93|     48|    0|
|   7777|null| 56|4939.32|   1233| null|
|   8888|null| 18|  78.29|    951| null|
|   9947|null| 66|3400.21|    687| null|
+-------+----+---+-------+-------+-----+



The resulting table is exactly the result that we set out to find!